# Make features (3rd.) including weekly ones

### Imports

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

import time
import sys
import gc
import pickle
import platform

### Load Data

In [2]:
train = pd.read_csv(os.path.abspath("../dat")+'/train.csv', encoding='cp949')
test = pd.read_csv(os.path.abspath("../dat")+'/test.csv',  encoding='cp949')

### Feature Engineering

주 단위 feature를 만들기 위해 week 필드 생성

In [3]:
train["week"] = train["date"].apply(lambda x : (x%100-1)//5+1 if (x%100-1)//5<6 else 6)

#### Transactions to monthly sales

*유의사항) Test Data에 있는 책 중에 1014개는 Train Data에는 없음.*

In [4]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len(test)

(1014, 13481)

월별 도서별 판매량 형식으로 데이터 변환

In [5]:
df = train.groupby(['month','item_id'])['count'].agg([('item_cnt_month', 'sum')]).reset_index()

In [6]:
for i in range(1,7):
    itemweek = train.query("week==@i").groupby(['month','item_id'])['count'].agg([(f'item_cnt_week{i}', 'sum')]).reset_index()
    df = pd.merge(df,itemweek,on=["month","item_id"],how="left")
df = df.fillna(0)

target value(월별 도서 판매량)를 최대 20으로 조정

In [7]:
df['item_cnt_month'] = df['item_cnt_month'].clip(0,20)

#### Add a book category feature

In [8]:
df = pd.merge(df, 
              train.drop_duplicates(subset='item_id').loc[:,['item_id', 'category']], 
              on='item_id', how='left')

#### Append test set

학습 데이터와 같은 feature를 만들기 위해 병합

In [9]:
test["month"] = 12
df = pd.concat([df, test], ignore_index=True, sort=False).fillna(0)

#### Add lag features

시차변수(1개월, 2개월, 3개월, 4개월 전 판매량) 생성

In [10]:
def lag_feature(df, lags, col):
    tmp = df[['month','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['month','item_id', col+f'_lag_{i}']
        shifted['month'] += i
        df = pd.merge(df, shifted, on=['month','item_id'], how='left')
    return df

In [11]:
df = lag_feature(df, [1,2,3,4], 'item_cnt_month')

In [12]:
for i in range(1,7):
    df = lag_feature(df,[1,2,3,4],f'item_cnt_week{i}')

월 평균 판매량

In [13]:
group = df.groupby(['month'])['item_cnt_month'].agg([('month_avg_item_cnt', 'mean')]).reset_index()
df = pd.merge(df, group, on=['month'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_avg_item_cnt')
df.drop(['month_avg_item_cnt'], axis=1, inplace=True)

In [14]:
for i in range(1,7):
    group = df.groupby(['month'])[f'item_cnt_week{i}'].agg([(f'week{i}_avg_item_cnt', 'mean')]).reset_index()
    df = pd.merge(df, group, on=['month'], how='left')
    df = lag_feature(df, [1,2,3,4], f'week{i}_avg_item_cnt')
    df.drop([f'week{i}_avg_item_cnt'], axis=1, inplace=True)

도서별 월평균 판매량

In [15]:
group = df.groupby(['month', 'item_id'])['item_cnt_month'].agg([('month_item_avg_item_cnt', 'mean')]).reset_index()
df = pd.merge(df, group, on=['month','item_id'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_item_avg_item_cnt')
df.drop(['month_item_avg_item_cnt'], axis=1, inplace=True)

In [16]:
for i in range(1,7):
    group = df.groupby(['month', 'item_id'])[f'item_cnt_week{i}'].agg([(f'week{i}_item_avg_item_cnt', 'mean')]).reset_index()
    df = pd.merge(df, group, on=['month','item_id'], how='left')
    df = lag_feature(df, [1,2,3,4], f'week{i}_item_avg_item_cnt')
    df.drop([f'week{i}_item_avg_item_cnt'], axis=1, inplace=True)

도서 분야별 월평균 판매량

In [17]:
group = df.groupby(['month', 'category']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_cat_avg_item_cnt']
group.reset_index(inplace=True)

group = df.groupby(['month', 'category'])['item_cnt_month'].agg([('month_cat_avg_item_cnt', 'mean')]).reset_index()

df = pd.merge(df, group, on=['month', 'category'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_cat_avg_item_cnt')
df.drop(['month_cat_avg_item_cnt'], axis=1, inplace=True)

In [18]:
for i in range(1,7):
    group = df.groupby(['month', 'category']).agg({f'item_cnt_week{i}': ['mean']})
    group.columns = [f'week{i}_date_cat_avg_item_cnt']
    group.reset_index(inplace=True)

    group = df.groupby(['month', 'category'])[f'item_cnt_week{i}'].agg([(f'week{i}_cat_avg_item_cnt', 'mean')]).reset_index()

    df = pd.merge(df, group, on=['month', 'category'], how='left')
    df = lag_feature(df, [1,2,3,4], f'week{i}_cat_avg_item_cnt')
    df.drop([f'week{i}_cat_avg_item_cnt'], axis=1, inplace=True)

In [19]:
df = df.drop(["item_cnt_week1","item_cnt_week2","item_cnt_week3","item_cnt_week4","item_cnt_week5","item_cnt_week6"],axis=1)

#### Final data preparation
Lag feature를 적용할 수 없는 1월부터 4월 데이터 제거

In [20]:
df = df[df.month > 4]

결측값 0으로 치환

In [21]:
def fill_na(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

df = fill_na(df)

도서 관련 feature 추가

In [22]:
book = pd.read_csv(os.path.abspath("../dat")+'/book_info.csv', encoding='cp949')
# book_info 의 pub_date의 오류로 보이는 데이터 변환 (인덱스 번호: 90867,493398)
book.loc[90867, 'pub_date'] = 201409
book.loc[49398, 'pub_date'] = 201505
book = book.fillna(0)
book["pub_date"] = book["pub_date"].astype(int)

In [23]:
book["pub_month"] = book["pub_date"].apply(lambda x: x%10000//100 if x//10000==2014 else 0)

In [24]:
df = pd.merge(df, book[["item_id","pub_month"]], on="item_id", how='left')

In [25]:
df["pub_binary"] = df["month"] > df["pub_month"]
df["pub_binary"] = df["pub_binary"].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154603 entries, 0 to 154602
Columns: 118 entries, month to pub_binary
dtypes: float64(113), int32(1), int64(3), object(1)
memory usage: 139.8+ MB


In [26]:
df

,month,item_id,item_cnt_month,category,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_4,item_cnt_week1_lag_1,item_cnt_week1_lag_2,item_cnt_week1_lag_3,item_cnt_week1_lag_4,item_cnt_week2_lag_1,item_cnt_week2_lag_2,item_cnt_week2_lag_3,item_cnt_week2_lag_4,item_cnt_week3_lag_1,item_cnt_week3_lag_2,item_cnt_week3_lag_3,item_cnt_week3_lag_4,item_cnt_week4_lag_1,item_cnt_week4_lag_2,item_cnt_week4_lag_3,item_cnt_week4_lag_4,item_cnt_week5_lag_1,item_cnt_week5_lag_2,item_cnt_week5_lag_3,item_cnt_week5_lag_4,item_cnt_week6_lag_1,item_cnt_week6_lag_2,item_cnt_week6_lag_3,item_cnt_week6_lag_4,month_avg_item_cnt_lag_1,month_avg_item_cnt_lag_2,month_avg_item_cnt_lag_3,month_avg_item_cnt_lag_4,week1_avg_item_cnt_lag_1,week1_avg_item_cnt_lag_2,week1_avg_item_cnt_lag_3,week1_avg_item_cnt_lag_4,week2_avg_item_cnt_lag_1,week2_avg_item_cnt_lag_2,week2_avg_item_cnt_lag_3,week2_avg_item_cnt_lag_4,week3_avg_item_cnt_lag_1,week3_avg_item_cnt_lag_2,week3_avg_item_cnt_lag_3,week3_avg_item_cnt_lag_4,week4_avg_item_cnt_lag_1,week4_avg_item_cnt_lag_2,...,week2_item_avg_item_cnt_lag_1,week2_item_avg_item_cnt_lag_2,week2_item_avg_item_cnt_lag_3,week2_item_avg_item_cnt_lag_4,week3_item_avg_item_cnt_lag_1,week3_item_avg_item_cnt_lag_2,week3_item_avg_item_cnt_lag_3,week3_item_avg_item_cnt_lag_4,week4_item_avg_item_cnt_lag_1,week4_item_avg_item_cnt_lag_2,week4_item_avg_item_cnt_lag_3,week4_item_avg_item_cnt_lag_4,week5_item_avg_item_cnt_lag_1,week5_item_avg_item_cnt_lag_2,week5_item_avg_item_cnt_lag_3,week5_item_avg_item_cnt_lag_4,week6_item_avg_item_cnt_lag_1,week6_item_avg_item_cnt_lag_2,week6_item_avg_item_cnt_lag_3,week6_item_avg_item_cnt_lag_4,month_cat_avg_item_cnt_lag_1,month_cat_avg_item_cnt_lag_2,month_cat_avg_item_cnt_lag_3,month_cat_avg_item_cnt_lag_4,week1_cat_avg_item_cnt_lag_1,week1_cat_avg_item_cnt_lag_2,week1_cat_avg_item_cnt_lag_3,week1_cat_avg_item_cnt_lag_4,week2_cat_avg_item_cnt_lag_1,week2_cat_avg_item_cnt_lag_2,week2_cat_avg_item_cnt_lag_3,week2_cat_avg_item_cnt_lag_4,week3_cat_avg_item_cnt_lag_1,week3_cat_avg_item_cnt_lag_2,week3_cat_avg_item_cnt_lag_3,week3_cat_avg_item_cnt_lag_4,week4_cat_avg_item_cnt_lag_1,week4_cat_avg_item_cnt_lag_2,week4_cat_avg_item_cnt_lag_3,week4_cat_avg_item_cnt_lag_4,week5_cat_avg_item_cnt_lag_1,week5_cat_avg_item_cnt_lag_2,week5_cat_avg_item_cnt_lag_3,week5_cat_avg_item_cnt_lag_4,week6_cat_avg_item_cnt_lag_1,week6_cat_avg_item_cnt_lag_2,week6_cat_avg_item_cnt_lag_3,week6_cat_avg_item_cnt_lag_4,pub_month,pub_binary
0,5,100000,1.0,잡지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,5,100004,1.0,전집,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.120691,0.0,0.0,0.0,0.419635,0.0,0.0,0.0,0.507441,0.0,0.0,0.0,0.4012,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,5,100030,1.0,잡지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,5,100038,1.0,청소년,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,...,0

pickle 형태로 분석에 사용할 최종 데이터 저장

In [29]:
df.to_pickle(os.path.abspath("../dat")+'/features_week.pkl')

# End